<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/tile_upscaler_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
!nvidia-smi
import os
import subprocess
import sys
import shutil
import re

# Проверяем, не установлено ли уже
if not os.path.exists('/content/installed_tile_upscaler'):
    print("=" * 70)
    print("📦 INSTALLING TILE UPSCALER")
    print("=" * 70)

    # Устанавливаем diffusers и связанные пакеты
    print("\n[1/4] Installing Python packages...")
    packages = [
        'diffusers',
        'transformers',
        'accelerate',
        'safetensors',
        'invisible-watermark',
        'huggingface_hub'
    ]

    for package in packages:
        print(f"  → {package}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("  ✓ Python packages installed")

    # Устанавливаем Real-ESRGAN
    print("\n[2/4] Installing Real-ESRGAN...")

    if not os.path.exists('/content/Real-ESRGAN'):
        print("  → Cloning repository...")
        get_ipython().system('git clone -q https://github.com/xinntao/Real-ESRGAN.git /content/Real-ESRGAN')

    os.chdir('/content/Real-ESRGAN')

    print("  → Installing dependencies...")
    get_ipython().system('pip install -q basicsr facexlib gfpgan -r requirements.txt')
    get_ipython().system('python setup.py develop > /dev/null 2>&1')

    print("  → Fixing compatibility issues...")

    # URL новой модели GFPGAN
    new_model_path = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'

    # Открываем файл inference_realesrgan.py и изменяем путь к модели
    inference_file = '/content/Real-ESRGAN/inference_realesrgan.py'
    if os.path.exists(inference_file):
        with open(inference_file, 'r') as f:
            script_content = f.read()

        # Изменяем путь к модели в строке скрипта
        new_script_content = re.sub(r"(model_path\s*=\s*[\"\']).*?([\"\'])", rf"\g<1>{new_model_path}\g<2>", script_content)

        with open(inference_file, 'w') as f:
            f.write(new_script_content)

    # Определяем версию Python
    python_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
    degradations_path = f'/usr/local/lib/{python_version}/dist-packages/basicsr/data/degradations.py'

    # Делаем бэкап
    if os.path.exists(degradations_path):
        get_ipython().system(f"cp -n {degradations_path} {degradations_path}.backup")

    # Фиксим импорт rgb_to_grayscale
    get_ipython().system(f"sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' {degradations_path}")

    # Записываем полный код degradations
    degradations_code = '''import cv2
import math
import numpy as np
import random
import torch
from torch.nn import functional as F

def random_add_gaussian_noise_pt(img, sigma_range=(0, 1.0), gray_prob=0, noise_gray_prob=0, clip=True, rounds=False):
    noise_sigma = random.uniform(*sigma_range)

    if random.random() < gray_prob:
        img = rgb_to_grayscale(img)

    if random.random() < noise_gray_prob:
        noise = torch.randn(*img.shape[1:], device=img.device) * noise_sigma
        noise = noise.unsqueeze(0).repeat(img.shape[0], 1, 1)
    else:
        noise = torch.randn_like(img) * noise_sigma

    out = img + noise

    if clip and rounds:
        out = torch.clamp((out * 255.0).round(), 0, 255) / 255.
    elif clip:
        out = torch.clamp(out, 0, 1)
    elif rounds:
        out = (out * 255.0).round() / 255.

    return out

def random_add_poisson_noise_pt(img, scale_range=(0, 1.0), gray_prob=0, clip=True, rounds=False):
    scale = random.uniform(*scale_range)

    if random.random() < gray_prob:
        img = rgb_to_grayscale(img)

    noise = torch.poisson(img * scale) / scale - img
    out = img + noise

    if clip and rounds:
        out = torch.clamp((out * 255.0).round(), 0, 255) / 255.
    elif clip:
        out = torch.clamp(out, 0, 1)
    elif rounds:
        out = (out * 255.0).round() / 255.

    return out

def rgb_to_grayscale(img):
    if img.shape[0] != 3:
        raise ValueError('Input image must have 3 channels')
    rgb_weights = torch.tensor([0.2989, 0.5870, 0.1140], device=img.device)
    grayscale = torch.sum(img * rgb_weights.view(-1, 1, 1), dim=0, keepdim=True)
    return grayscale

def circular_lowpass_kernel(cutoff, kernel_size, pad_to=0):
    if pad_to == 0:
        pad_to = kernel_size
    assert pad_to >= kernel_size, 'Pad size must be larger than kernel size'

    def _scaled_sinc(x):
        if x == 0:
            return torch.tensor(1.)
        x = x * math.pi
        return torch.sin(x) / x

    half_size = (kernel_size - 1) / 2.
    grid = torch.linspace(-half_size, half_size, kernel_size)
    x, y = torch.meshgrid(grid, grid)
    dist = torch.sqrt(x**2 + y**2)

    kernel = _scaled_sinc(dist * cutoff)
    kernel = kernel / kernel.sum()

    if pad_to > kernel_size:
        pad = (pad_to - kernel_size) // 2
        kernel = F.pad(kernel, [pad] * 4)

    return kernel

def random_mixed_kernels(
        kernel_list,
        kernel_prob,
        kernel_size=21,
        blur_sigma=0.1,
        blur_sigma_min=0.1,
        blur_sigma_max=10.0,
        blur_kernel_size=21,
        pad_to=0):
    num_kernels = len(kernel_list)
    kernel_type = np.random.choice(kernel_list, p=kernel_prob)

    if pad_to == 0:
        pad_to = kernel_size

    if kernel_type == 'iso':
        sigma = np.random.uniform(blur_sigma_min, blur_sigma_max)
        kernel = _generate_isotropic_gaussian_kernel(kernel_size, sigma, pad_to)
    elif kernel_type == 'aniso':
        sigma_x = np.random.uniform(blur_sigma_min, blur_sigma_max)
        sigma_y = np.random.uniform(blur_sigma_min, blur_sigma_max)
        rotation = np.random.uniform(-np.pi, np.pi)
        kernel = _generate_anisotropic_gaussian_kernel(kernel_size, sigma_x, sigma_y, rotation, pad_to)
    else:  # general
        kernel = circular_lowpass_kernel(blur_sigma, kernel_size, pad_to)

    return kernel

def _generate_isotropic_gaussian_kernel(kernel_size=21, sigma=0.1, pad_to=0):
    if pad_to == 0:
        pad_to = kernel_size

    half_size = (kernel_size - 1) / 2.
    grid = torch.linspace(-half_size, half_size, kernel_size)
    x, y = torch.meshgrid(grid, grid)
    kernel = torch.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel = kernel / kernel.sum()

    if pad_to > kernel_size:
        pad = (pad_to - kernel_size) // 2
        kernel = F.pad(kernel, [pad] * 4)

    return kernel

def _generate_anisotropic_gaussian_kernel(kernel_size=21, sigma_x=0.1, sigma_y=0.1, rotation=0, pad_to=0):
    if pad_to == 0:
        pad_to = kernel_size

    half_size = (kernel_size - 1) / 2.
    grid = torch.linspace(-half_size, half_size, kernel_size)
    x, y = torch.meshgrid(grid, grid)

    cos_theta = torch.cos(torch.tensor(rotation))
    sin_theta = torch.sin(torch.tensor(rotation))
    x_rot = cos_theta * x - sin_theta * y
    y_rot = sin_theta * x + cos_theta * y

    kernel = torch.exp(-(x_rot**2 / (2 * sigma_x**2) + y_rot**2 / (2 * sigma_y**2)))
    kernel = kernel / kernel.sum()

    if pad_to > kernel_size:
        pad = (pad_to - kernel_size) // 2
        kernel = F.pad(kernel, [pad] * 4)

    return kernel
'''

    with open(degradations_path, 'w') as f:
        f.write(degradations_code)

    print("  ✓ Real-ESRGAN installed")

    # Создаем структуру папок для Tile-Upscaler
    print("\n[3/4] Creating project structure...")

    if not os.path.exists('/content/Tile-Upscaler'):
        get_ipython().system('git clone -q https://github.com/gokayfem/Tile-Upscaler.git /content/Tile-Upscaler')

    os.chdir('/content/Tile-Upscaler')

    for folder in ['models/ControlNet', 'models/models/Stable-diffusion', 'models/VAE',
                   'models/embeddings', 'models/Lora', 'models/upscalers']:
        os.makedirs(folder, exist_ok=True)
    print("  ✓ Directories created")

    # Скачиваем модели используя hf_hub_download
    print("\n[4/4] Downloading AI models from Hugging Face...")
    print("  (This will take 5-10 minutes)")

    from huggingface_hub import hf_hub_download

    models = {
        "Stable Diffusion Model": {
            "repo_id": "dantea1118/juggernaut_reborn",
            "filename": "juggernaut_reborn.safetensors",
            "local_dir": "models/models/Stable-diffusion"
        },
        "RealESRGAN x2": {
            "repo_id": "ai-forever/Real-ESRGAN",
            "filename": "RealESRGAN_x2.pth",
            "local_dir": "models/upscalers/"
        },
        "RealESRGAN x4": {
            "repo_id": "ai-forever/Real-ESRGAN",
            "filename": "RealESRGAN_x4.pth",
            "local_dir": "models/upscalers/"
        },
        "Embedding (BadPrompt)": {
            "repo_id": "philz1337x/embeddings",
            "filename": "verybadimagenegative_v1.3.pt",
            "local_dir": "models/embeddings"
        },
        "Embedding (Juggernaut Neg)": {
            "repo_id": "philz1337x/embeddings",
            "filename": "JuggernautNegative-neg.pt",
            "local_dir": "models/embeddings"
        },
        "LoRA (SDXLrender)": {
            "repo_id": "philz1337x/loras",
            "filename": "SDXLrender_v2.0.safetensors",
            "local_dir": "models/Lora"
        },
        "LoRA (More Details)": {
            "repo_id": "philz1337x/loras",
            "filename": "more_details.safetensors",
            "local_dir": "models/Lora"
        },
        "ControlNet Tile": {
            "repo_id": "lllyasviel/ControlNet-v1-1",
            "filename": "control_v11f1e_sd15_tile.pth",
            "local_dir": "models/ControlNet"
        },
        "VAE": {
            "repo_id": "stabilityai/sd-vae-ft-mse-original",
            "filename": "vae-ft-mse-840000-ema-pruned.safetensors",
            "local_dir": "models/VAE"
        }
    }

    failed_downloads = []

    for model_name, model_info in models.items():
        file_path = os.path.join(model_info["local_dir"], model_info["filename"])

        if not os.path.exists(file_path):
            print(f"\n  → Downloading {model_name}...")
            try:
                hf_hub_download(
                    repo_id=model_info["repo_id"],
                    filename=model_info["filename"],
                    local_dir=model_info["local_dir"]
                )
                if os.path.exists(file_path):
                    size_mb = os.path.getsize(file_path) / (1024*1024)
                    print(f"    ✓ Downloaded ({size_mb:.1f} MB)")
                else:
                    print(f"    ✗ Download failed")
                    failed_downloads.append(model_name)
            except Exception as e:
                print(f"    ✗ Error: {str(e)}")
                failed_downloads.append(model_name)
        else:
            size_mb = os.path.getsize(file_path) / (1024*1024)
            print(f"  ✓ {model_name} already exists ({size_mb:.1f} MB)")

    # Проверка результатов
    print("\n" + "=" * 70)
    print("VERIFYING INSTALLATION")
    print("=" * 70)

    all_files = [
        'models/ControlNet/control_v11f1e_sd15_tile.pth',
        'models/models/Stable-diffusion/juggernaut_reborn.safetensors',
        'models/VAE/vae-ft-mse-840000-ema-pruned.safetensors',
        'models/embeddings/verybadimagenegative_v1.3.pt',
        'models/embeddings/JuggernautNegative-neg.pt',
        'models/Lora/SDXLrender_v2.0.safetensors',
        'models/Lora/more_details.safetensors',
        'models/upscalers/RealESRGAN_x2.pth',
        'models/upscalers/RealESRGAN_x4.pth'
    ]

    missing_files = []
    total_size = 0

    print("\nInstalled files:")
    for file in all_files:
        if os.path.exists(file):
            size_mb = os.path.getsize(file) / (1024*1024)
            total_size += size_mb
            print(f"  ✓ {os.path.basename(file)} ({size_mb:.1f} MB)")
        else:
            print(f"  ✗ {os.path.basename(file)} - MISSING")
            missing_files.append(file)

    print(f"\nTotal size: {total_size:.1f} MB")

    if missing_files or failed_downloads:
        print("\n" + "=" * 70)
        print("⚠️  INSTALLATION INCOMPLETE")
        print("=" * 70)
        if failed_downloads:
            print("\nFailed downloads:")
            for item in failed_downloads:
                print(f"  • {item}")
        if missing_files:
            print("\nMissing files:")
            for file in missing_files:
                print(f"  • {os.path.basename(file)}")
        print("\n→ Please run this cell again to retry downloading missing files.")
        print("=" * 70)
    else:
        # Создаем маркер успешной установки
        open('/content/installed_tile_upscaler', 'w').close()
        print("\n" + "=" * 70)
        print("✅ INSTALLATION COMPLETED SUCCESSFULLY!")
        print("=" * 70)
        print("\nYou can now proceed to:")
        print("  → Cell 2: Upload your image")
        print("  → Cell 3: Configure settings")
        print("  → Cell 4: Process image")
        print("  → Cell 5: Visualize results")
        print("  → Cell 6: Download result")
        print("=" * 70)
else:
    print("=" * 70)
    print("✅ ALREADY INSTALLED")
    print("=" * 70)
    os.chdir('/content/Tile-Upscaler')

    # Показываем информацию о установленных моделях
    print("\nInstalled models:")
    all_files = [
        'models/ControlNet/control_v11f1e_sd15_tile.pth',
        'models/models/Stable-diffusion/juggernaut_reborn.safetensors',
        'models/VAE/vae-ft-mse-840000-ema-pruned.safetensors',
        'models/embeddings/verybadimagenegative_v1.3.pt',
        'models/embeddings/JuggernautNegative-neg.pt',
        'models/Lora/SDXLrender_v2.0.safetensors',
        'models/Lora/more_details.safetensors',
        'models/upscalers/RealESRGAN_x2.pth',
        'models/upscalers/RealESRGAN_x4.pth'
    ]

    total_size = 0
    for file in all_files:
        if os.path.exists(file):
            size_mb = os.path.getsize(file) / (1024*1024)
            total_size += size_mb
            print(f"  ✓ {os.path.basename(file)} ({size_mb:.1f} MB)")

    print(f"\nTotal size: {total_size:.1f} MB")
    print("\n→ To reinstall: !rm /content/installed_tile_upscaler")
    print("=" * 70)

# Добавляем пути
if '/content/Real-ESRGAN' not in sys.path:
    sys.path.insert(0, '/content/Real-ESRGAN')

print("\n✓ Environment ready")

In [ ]:
#@title ##**Upload Image** { display-mode: "form" }

from google.colab import files
from PIL import Image
import io

print("📁 Please select an image to upload...")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    input_image = Image.open(io.BytesIO(uploaded[filename]))

    input_image.save('/content/input_image.png')

    print(f"✅ Image uploaded successfully: {filename}")
    print(f"   Size: {input_image.size}")

    #display(input_image)
else:
    print("❌ No image uploaded!")

In [ ]:
#@title ##**Configuration** { display-mode: "form" }

#@markdown ### Output Settings
resolution = 2048 #@param {type:"slider", min:512, max:4096, step:128}

#@markdown ### Generation Settings
num_inference_steps = 50 #@param {type:"slider", min:10, max:100, step:5}
strength = 0.3 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
guidance_scale = 3 #@param {type:"slider", min:1, max:20, step:0.5}
controlnet_strength = 0.2 #@param {type:"slider", min:0.0, max:2.0, step:0.1}

#@markdown ### Effects
hdr_effect = 0 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
face_enhance = True #@param {type:"boolean"}

#@markdown ### Scheduler
scheduler = "DPM++ 3M SDE Karras" #@param ["DDIM", "DPM++ 3M SDE Karras", "DPM++ 3M Karras"]

#@markdown ### Prompt (Optional)
custom_prompt = "photo" #@param {type:"string"}
custom_negative_prompt = "" #@param {type:"string"}

print("⚙️ Configuration saved!")
print(f"Resolution: {resolution}")
print(f"Steps: {num_inference_steps}")
print(f"Strength: {strength}")
print(f"Guidance Scale: {guidance_scale}")
print(f"ControlNet Strength: {controlnet_strength}")
print(f"HDR Effect: {hdr_effect}")
print(f"Face Enhancement: {'Enabled' if face_enhance else 'Disabled'}")
print(f"Scheduler: {scheduler}")

In [ ]:
#@title ##**Run** { display-mode: "form" }

import os
import sys
import time
import torch
import warnings
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, DDIMScheduler, DPMSolverMultistepScheduler
from diffusers.models import AutoencoderKL
from PIL import Image
import cv2
import numpy as np
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
import random
import math
from tqdm.auto import tqdm

# Подавляем предупреждения
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

os.chdir('/content/Tile-Upscaler')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_scheduler(scheduler_name, config):
    if scheduler_name == "DDIM":
        return DDIMScheduler.from_config(config)
    elif scheduler_name == "DPM++ 3M SDE Karras":
        return DPMSolverMultistepScheduler.from_config(config, algorithm_type="sde-dpmsolver++", use_karras_sigmas=True)
    elif scheduler_name == "DPM++ 3M Karras":
        return DPMSolverMultistepScheduler.from_config(config, algorithm_type="dpmsolver++", use_karras_sigmas=True)
    else:
        raise ValueError(f"Unknown scheduler: {scheduler_name}")

class LazyRealESRGAN:
    def __init__(self, device, scale):
        self.device = device
        self.scale = scale
        self.model = None

    def load_model(self):
        if self.model is None:
            model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=self.scale)
            model_path = f'models/upscalers/RealESRGAN_x{self.scale}.pth'

            self.model = RealESRGANer(
                scale=self.scale,
                model_path=model_path,
                model=model,
                tile=512,
                tile_pad=10,
                pre_pad=0,
                half=True if 'cuda' in str(self.device) else False,
                device=self.device
            )

    def predict(self, img):
        self.load_model()
        img_np = np.array(img)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
        output, _ = self.model.enhance(img_bgr, outscale=self.scale)
        output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
        return Image.fromarray(output_rgb)

class LazyGFPGAN:
    def __init__(self, device):
        self.device = device
        self.model = None

    def load_model(self):
        if self.model is None:
            from gfpgan import GFPGANer

            model_path = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'

            self.model = GFPGANer(
                model_path=model_path,
                upscale=1,
                arch='clean',
                channel_multiplier=2,
                bg_upsampler=None,
                device=self.device
            )

    def enhance(self, img):
        self.load_model()
        img_np = np.array(img)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
        _, _, output = self.model.enhance(img_bgr, has_aligned=False, only_center_face=False, paste_back=True)
        output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
        return Image.fromarray(output_rgb)

# Инициализация моделей
if 'lazy_realesrgan_x2' not in globals():
    lazy_realesrgan_x2 = LazyRealESRGAN(device, scale=2)
    lazy_realesrgan_x4 = LazyRealESRGAN(device, scale=4)

if 'lazy_gfpgan' not in globals():
    lazy_gfpgan = LazyGFPGAN(device)

if 'pipe' not in globals():
    print("🔄 Loading models...")

    # Подавляем вывод от diffusers
    from diffusers.utils import logging as diffusers_logging
    diffusers_logging.set_verbosity_error()

    controlnet = ControlNetModel.from_single_file(
        "models/ControlNet/control_v11f1e_sd15_tile.pth", torch_dtype=torch.float16
    )

    pipe = StableDiffusionControlNetImg2ImgPipeline.from_single_file(
        "models/models/Stable-diffusion/juggernaut_reborn.safetensors",
        controlnet=controlnet,
        torch_dtype=torch.float16,
        use_safetensors=True,
        safety_checker=None
    )

    vae = AutoencoderKL.from_single_file(
        "models/VAE/vae-ft-mse-840000-ema-pruned.safetensors",
        torch_dtype=torch.float16
    )
    pipe.vae = vae

    pipe.load_textual_inversion("models/embeddings/verybadimagenegative_v1.3.pt")
    pipe.load_textual_inversion("models/embeddings/JuggernautNegative-neg.pt")
    pipe.load_lora_weights("models/Lora/SDXLrender_v2.0.safetensors")
    pipe.fuse_lora(lora_scale=0.5)
    pipe.load_lora_weights("models/Lora/more_details.safetensors")
    pipe.fuse_lora(lora_scale=1.)

    pipe.enable_freeu(s1=0.9, s2=0.2, b1=1.3, b2=1.4)
    pipe.to(device)

    # Отключаем прогресс-бар для pipeline
    pipe.set_progress_bar_config(disable=True)

    print("✅ Models loaded!")

# Устанавливаем выбранный scheduler
pipe.scheduler = get_scheduler(scheduler, pipe.scheduler.config)
pipe.set_progress_bar_config(disable=True)

def progressive_upscale(input_image, target_resolution, steps=3):
    current_image = input_image.convert("RGB")
    current_size = max(current_image.size)

    for _ in range(steps):
        if current_size >= target_resolution:
            break

        scale_factor = min(2, target_resolution / current_size)
        new_size = (int(current_image.width * scale_factor), int(current_image.height * scale_factor))

        if scale_factor <= 1.5:
            current_image = current_image.resize(new_size, Image.LANCZOS)
        else:
            current_image = lazy_realesrgan_x2.predict(current_image)

        current_size = max(current_image.size)

    if current_size != target_resolution:
        aspect_ratio = current_image.width / current_image.height
        if current_image.width > current_image.height:
            new_size = (target_resolution, int(target_resolution / aspect_ratio))
        else:
            new_size = (int(target_resolution * aspect_ratio), target_resolution)
        current_image = current_image.resize(new_size, Image.LANCZOS)

    return current_image

def create_hdr_effect(original_image, hdr):
    if hdr == 0:
        return original_image
    cv_original = cv2.cvtColor(np.array(original_image), cv2.COLOR_RGB2BGR)
    factors = [1.0 - 0.9 * hdr, 1.0 - 0.7 * hdr, 1.0 - 0.45 * hdr,
              1.0 - 0.25 * hdr, 1.0, 1.0 + 0.2 * hdr,
              1.0 + 0.4 * hdr, 1.0 + 0.6 * hdr, 1.0 + 0.8 * hdr]
    images = [cv2.convertScaleAbs(cv_original, alpha=factor) for factor in factors]
    merge_mertens = cv2.createMergeMertens()
    hdr_image = merge_mertens.process(images)
    hdr_image_8bit = np.clip(hdr_image * 255, 0, 255).astype('uint8')
    return Image.fromarray(cv2.cvtColor(hdr_image_8bit, cv2.COLOR_BGR2RGB))

def prepare_image(input_image, resolution, hdr):
    upscaled_image = progressive_upscale(input_image, resolution)
    return create_hdr_effect(upscaled_image, hdr)

def create_gaussian_weight(tile_size, sigma=0.3):
    x = np.linspace(-1, 1, tile_size)
    y = np.linspace(-1, 1, tile_size)
    xx, yy = np.meshgrid(x, y)
    gaussian_weight = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    return gaussian_weight

def adaptive_tile_size(image_size, base_tile_size=512, max_tile_size=1024):
    w, h = image_size
    aspect_ratio = w / h
    if aspect_ratio > 1:
        tile_w = min(w, max_tile_size)
        tile_h = min(int(tile_w / aspect_ratio), max_tile_size)
    else:
        tile_h = min(h, max_tile_size)
        tile_w = min(int(tile_h * aspect_ratio), max_tile_size)
    return max(tile_w, base_tile_size), max(tile_h, base_tile_size)

def process_tile(tile, num_inference_steps, strength, guidance_scale, controlnet_strength):
    prompt = custom_prompt if custom_prompt else "masterpiece, best quality, highres"
    negative_prompt = custom_negative_prompt if custom_negative_prompt else "low quality, normal quality, ugly, blurry, blur, lowres, bad anatomy, bad hands, cropped, worst quality, verybadimagenegative_v1.3, JuggernautNegative-neg"

    options = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "image": tile,
        "control_image": tile,
        "num_inference_steps": num_inference_steps,
        "strength": strength,
        "guidance_scale": guidance_scale,
        "controlnet_conditioning_scale": float(controlnet_strength),
        "generator": torch.Generator(device=device).manual_seed(random.randint(0, 2147483647)),
    }

    return np.array(pipe(**options).images[0])

# Загружаем входное изображение
input_image = Image.open('/content/input_image.png')

print("🎨 Processing image...")
start_time = time.time()

torch.cuda.empty_cache()

# Подготовка изображения
condition_image = prepare_image(input_image, resolution, hdr_effect)
W, H = condition_image.size

# Adaptive tiling
tile_width, tile_height = adaptive_tile_size((W, H))

overlap = min(64, tile_width // 8, tile_height // 8)
num_tiles_x = math.ceil((W - overlap) / (tile_width - overlap))
num_tiles_y = math.ceil((H - overlap) / (tile_height - overlap))

result = np.zeros((H, W, 3), dtype=np.float32)
weight_sum = np.zeros((H, W, 1), dtype=np.float32)

gaussian_weight = create_gaussian_weight(max(tile_width, tile_height))

total_tiles = num_tiles_x * num_tiles_y

# Один прогресс-бар для всех тайлов
with tqdm(total=total_tiles, desc="Processing tiles", unit="tile") as pbar:
    for i in range(num_tiles_y):
        for j in range(num_tiles_x):
            left = j * (tile_width - overlap)
            top = i * (tile_height - overlap)
            right = min(left + tile_width, W)
            bottom = min(top + tile_height, H)

            current_tile_size = (bottom - top, right - left)

            tile = condition_image.crop((left, top, right, bottom))
            tile = tile.resize((tile_width, tile_height))

            result_tile = process_tile(tile, num_inference_steps, strength, guidance_scale, controlnet_strength)

            if current_tile_size != (tile_width, tile_height):
                result_tile = cv2.resize(result_tile, current_tile_size[::-1])
                tile_weight = cv2.resize(gaussian_weight, current_tile_size[::-1])
            else:
                tile_weight = gaussian_weight[:current_tile_size[0], :current_tile_size[1]]

            result[top:bottom, left:right] += result_tile * tile_weight[:, :, np.newaxis]
            weight_sum[top:bottom, left:right] += tile_weight[:, :, np.newaxis]

            pbar.update(1)

final_result = (result / weight_sum).astype(np.uint8)
output_image = Image.fromarray(final_result)

# Применяем GFPGAN для улучшения лиц если включено
if face_enhance:
    print("👤 Enhancing faces...")
    output_image = lazy_gfpgan.enhance(output_image)

# Сохраняем результат
output_image.save('/content/output_image.png')

end_time = time.time()
print(f"✅ Done in {end_time - start_time:.1f}s | Size: {output_image.size}")

In [ ]:
#@title ##**Visualize** { display-mode: "form" }

import os
import PIL.Image
import numpy as np
from IPython.display import HTML, display
import base64
from io import BytesIO

#@markdown ### Visualization Method
visualization_method = "Slider" #@param ["Side-by-Side", "Slider"]

def resize_image_maintain_aspect(image, max_width, target_height=None):
    width, height = image.size
    if width > max_width:
        new_height = int(height * max_width / width)
        image = image.resize((max_width, new_height), PIL.Image.LANCZOS)
    if target_height is not None and image.size[1] != target_height:
        new_width = int(image.size[0] * target_height / image.size[1])
        image = image.resize((new_width, target_height), PIL.Image.LANCZOS)
    return image

def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Загружаем изображения
if not os.path.exists('/content/input_image.png') or not os.path.exists('/content/output_image.png'):
    print("❌ Error: Images not found. Please run the processing step first.")
else:
    image_original = PIL.Image.open('/content/input_image.png')
    image_restore = PIL.Image.open('/content/output_image.png')

    if visualization_method == "Side-by-Side":
        max_width = 500
        image_original = resize_image_maintain_aspect(image_original, max_width)
        image_restore = resize_image_maintain_aspect(image_restore, max_width)
        target_height = min(image_original.size[1], image_restore.size[1])
        image_original = resize_image_maintain_aspect(image_original, max_width, target_height)
        image_restore = resize_image_maintain_aspect(image_restore, max_width, target_height)

        combined_width = image_original.size[0] + image_restore.size[0]
        combined_image = PIL.Image.new('RGB', (combined_width, target_height))
        combined_image.paste(image_original, (0, 0))
        combined_image.paste(image_restore, (image_original.size[0], 0))
        display(combined_image)

    else:  # Slider
        max_width = min(image_restore.size[0], 1000)
        image_restore = resize_image_maintain_aspect(image_restore, max_width)
        target_height = image_restore.size[1]
        image_original = resize_image_maintain_aspect(image_original, max_width, target_height)

        if image_original.mode != 'RGB':
            image_original = image_original.convert('RGB')
        if image_restore.mode != 'RGB':
            image_restore = image_restore.convert('RGB')

        original_base64 = image_to_base64(image_original)
        restore_base64 = image_to_base64(image_restore)

        html_code = f"""
        <div style="position: relative; width: {image_restore.size[0]}px; height: {image_restore.size[1]}px; margin: 20px auto;">
            <div style="position: relative; width: 100%; height: 100%; overflow: hidden;">
                <img src="data:image/png;base64,{original_base64}" style="position: absolute; width: 100%; height: 100%;">
                <div style="position: absolute; width: 100%; height: 100%; overflow: hidden; clip-path: inset(0 0 0 50%);">
                    <img src="data:image/png;base64,{restore_base64}" style="position: absolute; width: 100%; height: 100%;">
                </div>
            </div>
            <div class="slider" style="position: absolute; top: 0; bottom: 0; width: 4px; background: white; cursor: ew-resize; left: 50%; box-shadow: 0 0 5px rgba(0,0,0,0.5);">
                <div style="position: absolute; top: 50%; transform: translateY(-50%); width: 20px; height: 20px; background: white; border-radius: 50%; left: -8px;"></div>
            </div>
        </div>
        <script>
            (function() {{
                const slider = document.querySelector('.slider');
                if (!slider) return;

                let isDragging = false;
                const container = slider.parentElement.querySelector('div:nth-child(1)');
                const clipDiv = container.querySelector('div');

                slider.addEventListener('mousedown', (e) => {{
                    isDragging = true;
                    e.preventDefault();
                }});

                document.addEventListener('mouseup', () => {{
                    isDragging = false;
                }});

                document.addEventListener('mousemove', (e) => {{
                    if (!isDragging) return;
                    const rect = container.getBoundingClientRect();
                    let x = e.clientX - rect.left;
                    if (x < 0) x = 0;
                    if (x > rect.width) x = rect.width;
                    const percentage = (x / rect.width) * 100;
                    slider.style.left = percentage + '%';
                    clipDiv.style.clipPath = `inset(0 0 0 ${{percentage}}%)`;
                }});

                slider.addEventListener('touchstart', (e) => {{
                    isDragging = true;
                    e.preventDefault();
                }});

                document.addEventListener('touchend', () => {{
                    isDragging = false;
                }});

                document.addEventListener('touchmove', (e) => {{
                    if (!isDragging) return;
                    const rect = container.getBoundingClientRect();
                    let x = e.touches[0].clientX - rect.left;
                    if (x < 0) x = 0;
                    if (x > rect.width) x = rect.width;
                    const percentage = (x / rect.width) * 100;
                    slider.style.left = percentage + '%';
                    clipDiv.style.clipPath = `inset(0 0 0 ${{percentage}}%)`;
                }});
            }})();
        </script>
        """

        display(HTML(html_code))

In [ ]:
#@title ##**Download** { display-mode: "form" }

from google.colab import files

if os.path.exists('/content/output_image.png'):
    print("💾 Downloading upscaled image...")
    files.download('/content/output_image.png')
    print("✅ Download started!")
else:
    print("❌ No output image found! Please run the processing step first.")